## Parse IFC file

In [1]:
import re

import ifcopenshell as ifc

In [2]:
bim = ifc.open('../assets/private/histo_v2.ifc')

In [3]:
from ifcopenshell import geom as ifc_geom
from ifcopenshell.util import shape as ifc_shape

import pandas as pd

settings = ifc_geom.settings()
settings.set(settings.USE_WORLD_COORDS, True)  # Find global coordinates

def get_area(x):
    shape = ifc_geom.create_shape(settings, x)
    geom = shape.geometry
    return ifc_shape.get_footprint_area(geom)

def get_centroid(x):
    shape = ifc_geom.create_shape(settings, x)
    geom = shape.geometry
    x, y, z = ifc_shape.get_bbox_centroid(geom)
    z = ifc_shape.get_shape_bottom_elevation(shape, geom)
    return (x, y)

def get_bottom_elevation(x):
    shape = ifc_geom.create_shape(settings, x)
    geom = shape.geometry
    z = ifc_shape.get_shape_bottom_elevation(shape, geom)
    return z

spaces = [
    x for x in bim.by_type('IfcSpace') if x.Description is not None
]

data = [
    {
        'id': x.id(),
        'description': re.sub(r'\s+', ' ', x.Description).lower(),
        'area': get_area(x),
        'centroid': get_centroid(x),
        'elevation': get_bottom_elevation(x)
    }
    for x in spaces
]

df = pd.DataFrame(data)
df

,id,description,area,centroid,elevation
0,4446,digital pathology,12.651152,"(87.03472352338409, 56.17319426046846)",7.0
1,4698,cupboard (2),5.490400,"(52.60837679466721, 57.86986107222328)",10.5
2,4722,cupboard (2),3.311260,"(55.09380988963899, 61.26736107222324)",10.5
3,4746,white room cut-up,26.102966,"(69.29587679466721, 63.49391196897473)",10.5
4,4770,cupboard (1),0.883575,"(54.489626794667224, 58.45486107222329)",10.5
5,4866,filing room,14.239375,"(49.868376794667206, 56.221111072223266)",10.5
6,4919,ladies toilet,10.598825,"(56.582126794667204, 63.99736107222324)",10.5
7,4943,ladies toilet,6.074083,"(57.6821267946672, 63.07611107222323)",10.5
8,4967,cupboard (1),0.494019,"(52.83712679466721, 61.74111107222325)",10.5
9,4991,main lab section leaders,16.412656,"(52.932126794667184, 63.54736107222324)",10.5


In [8]:
df.round(3).sort_values('area', ascending=False)

,id,description,area,centroid,elevation
10,5023,main lab,102.112,"(41.762126794667196, 59.87486107222327)",10.5
19,6010,processing room (embedding),42.976,"(78.93510577547727, 56.17236107222316)",10.5
21,6060,lilac room cut-up,41.988,"(65.2496267946672, 56.22111107222324)",10.5
11,5073,specimen storage,31.310,"(85.84729889245867, 63.5334604480077)",10.5
24,7266,specimen reception,29.160,"(61.164626794667214, 63.49391196897474)",10.5
15,5366,staining room,27.772,"(48.5071267946672, 63.499861072223254)",10.5
20,6036,yellow room cut-up,27.488,"(72.08962679466718, 56.22111107222323)",10.5
14,5244,green room cut-up,26.867,"(80.18117541283576, 63.53346044800773)",10.5
3,4746,white room cut-up,26.103,"(69.29587679466721, 63.49391196897473)",10.5
26,9201,access/egress,25.728,"(58.083376794667274, 56.78611107222305)",10.5
